# Edward源代码分析

那个inference对象的run封装了调用initialize生成计算图和迭代update的操作。
对于klqp来说，它的update,initialize逻辑继承自variational_inference。
其中update的核心逻辑是调用已生成的
计算图的优化op，该op是在initialize调用build_loss_and_gradients生成的。
这个方法在klqp被重载，绑定到该模块几个核心函数上了。
分别是对不同的概率结构生成不同的计算图。其中最一般的就是
build_score_loss_and_gradients方法。不过其实所有方法其实都用了Monte Carlo采样

贝叶斯模型虽然参数是随机变量，但如果参数的随机变量的分布又全是随机变量
就没完没了了。于是终究在超参数阶段归于确定参数。
那贝叶斯推断又在推断什么呢？既然我们已经以超参数给出了参数的分布？

给出的参数分布与之后它们怎么影响隐变量又怎么影响可观测变量的所有关系，
合成了联合分布函数或联合似然函数。那么我们就可以以可观测变量去预测
参数随机变量或隐状态的取值，就像X先在[0,1]均匀取一个值，
然后以此值为中心，加上噪声再生成10个值一样。这个模型本身是个联合分布，
给定数据后当然有信息可以推断出中心的分布。而此处设定时那个分布其实是
参数的先验分布，我们本来就是按参数的先验分布，给定参数时隐状态的条件分布，
给定隐状态和参数时可观测变量的条件分布来描述联合分布而不是直接给出联合分布的。
当然从推断的角度说，直接在先验分布上求极大似然或均值没什么意义，
因为这个分布终究会被条件分布和数据冲掉。

var_list里存的是qx式变量的超参数，优化过程比较隐晦，最重要的逻辑隐藏在
闷身发大财的copy函数，这个copy做了dict_swap变换计算图，才能使得数据
能够被引入，参数能够被优化。

构造计算图的核心代码是


```
def build_score_loss_and_gradients(inference, var_list):
  """Build loss function and gradients based on the score function
  estimator (Paisley et al., 2012).

  Computed by sampling from :math:`q(z;\lambda)` and evaluating the
  expectation using Monte Carlo sampling.
  """
  p_log_prob = [0.0] * inference.n_samples
  q_log_prob = [0.0] * inference.n_samples
  for s in range(inference.n_samples):
    # Form dictionary in order to replace conditioning on prior or
    # observed variable with conditioning on a specific value.
    scope = 'inference_' + str(id(inference)) + '/' + str(s)
    dict_swap = {}
    for x, qx in six.iteritems(inference.data):
      if isinstance(x, RandomVariable):
        if isinstance(qx, RandomVariable):
          qx_copy = copy(qx, scope=scope)
          dict_swap[x] = qx_copy.value()
        else:
          dict_swap[x] = qx

    for z, qz in six.iteritems(inference.latent_vars):
      # Copy q(z) to obtain new set of posterior samples.
      qz_copy = copy(qz, scope=scope)
      dict_swap[z] = qz_copy.value()
      q_log_prob[s] += tf.reduce_sum(
          inference.scale.get(z, 1.0) *
          qz_copy.log_prob(tf.stop_gradient(dict_swap[z])))

    for z in six.iterkeys(inference.latent_vars):
      z_copy = copy(z, dict_swap, scope=scope)
      p_log_prob[s] += tf.reduce_sum(
          inference.scale.get(z, 1.0) * z_copy.log_prob(dict_swap[z]))

    for x in six.iterkeys(inference.data):
      if isinstance(x, RandomVariable):
        x_copy = copy(x, dict_swap, scope=scope)
        p_log_prob[s] += tf.reduce_sum(
            inference.scale.get(x, 1.0) * x_copy.log_prob(dict_swap[x]))

  p_log_prob = tf.stack(p_log_prob)
  q_log_prob = tf.stack(q_log_prob)

  if inference.logging:
    summary_key = 'summaries_' + str(id(inference))
    tf.summary.scalar("loss/p_log_prob", tf.reduce_mean(p_log_prob),
                      collections=[summary_key])
    tf.summary.scalar("loss/q_log_prob", tf.reduce_mean(q_log_prob),
                      collections=[summary_key])

  losses = p_log_prob - q_log_prob
  loss = -tf.reduce_mean(losses)

  grads = tf.gradients(
      -tf.reduce_mean(q_log_prob * tf.stop_gradient(losses)),
      var_list)
  grads_and_vars = list(zip(grads, var_list))
  return loss, grads_and_vars

```

其中`var_list`是近似分布的超参数，被优化的tensorflow变量。`inference.data`是构建inference时传入的data字典，其key是模型（可观测）变量，
value是对应的数据。而`inference.latent_vars`是构建inference时传入的latent_vars字典（第一个参数），其指定了对哪些变量求近似后验分布,
key是原模型的变量，value是近似分布的变量。

这段代码构造的计算图之后每次inference.update都会计算一次，其核心在于`copy`函数，
该函数在带`dict_swap`调用时把原模型计算图里的模型变量换成了近似分布的变量，见下面的代码文档中的例子：

```
  Examples
  --------
  >>> x = tf.constant(2.0)
  >>> y = tf.constant(3.0)
  >>> z = x * y
  >>>
  >>> qx = tf.constant(4.0)
  >>> # The TensorFlow graph is currently
  >>> # `x` -> `z` <- y`, `qx`
  >>>
  >>> # This adds a subgraph with newly copied nodes,
  >>> # `qx` -> `copied/z` <- `copied/y`
  >>> z_new = ed.copy(z, {x: qx})
  >>>
  >>> sess = tf.Session()
  >>> sess.run(z)
  6.0
  >>> sess.run(z_new)
  12.0
  """
```

如此，我们分别查看损失函数的构造步骤，显然它想让p_log_prob最大化，可以看到

```
    for x in six.iterkeys(inference.data):
      if isinstance(x, RandomVariable):
        x_copy = copy(x, dict_swap, scope=scope)
        p_log_prob[s] += tf.reduce_sum(
            inference.scale.get(x, 1.0) * x_copy.log_prob(dict_swap[x]))

```

这段代码中`x`是可观测变量时，之前的代码会使得`dict_swap[x]`的值就是实际观测值。`x_copy`是从原模型计算图的模型变量`x`“copy”出来的。
但显然，若计算固定的原计算图中的`log_prob`，则这与我们想优化的参数没有建立任何关系。这里这个`copy`把原模型的随机变量`x`依赖的原模型的随机参数
换成了近似分布的随机参数。而近似分布的随机参数受近似分布的确定参数的控制，这些确定参数将被tensorflow以似然原则最优化。

但如果我们不只是想近似直接产生出数据的那些随机参数或隐变量的分布时，这一结果也是有用的：

```
    for z, qz in six.iteritems(inference.latent_vars):
      # Copy q(z) to obtain new set of posterior samples.
      qz_copy = copy(qz, scope=scope)
      dict_swap[z] = qz_copy.value()

    for z in six.iterkeys(inference.latent_vars):
      z_copy = copy(z, dict_swap, scope=scope)
      p_log_prob[s] += tf.reduce_sum(
          inference.scale.get(z, 1.0) * z_copy.log_prob(dict_swap[z]))
```

可以看到，求出最接近数据的那些随机参数或隐变量的近似分布后，我们可以调整决定这些随机参数的随机参数的近似分布，
去使得拟合这些“低层”的随机参数的边缘分布。而“顶层”参数去拟合“低层”参数（最底层的是观测数据）和拟合数据一样使用似然准则，
但这里不是用数据来计算对数似然而是用低层分布的采样值，显然，我们可以假设低层分布充分接近真实分布（因为低层训练不会受高层影响，
而最底层的训练是基于实际数据的，可以看做最底层充分接近真实分布后倒数第二层开始充分接近真实分布..以此类推），这么看有点像BP算法。

比如下图所示，隐变量X决定隐变量Y，Y决定可观测变量Z，Z有实现值dZ。则我们设近似Y的随机变量qY，
并改变qY的参数pY去使得Z合成qY而不是Y时产生dZ的似然度最大。充分训练后qY的分布将逼近Y给定dZ时的边缘条件分布。
然后我们采集qY足够多的样本dY来类似的训练近似X的随机变量qX。如此，最终qX，qY都会在我们给它们设定的分布下找到对真正的X,Y给定dZ的条件边缘分布。

